# Kryptografia asymetryczna - kryptosystem RSA 
Kryptografia asymetryczna charakteryzuje się wykorzystaniem **pary kluczy publiczny-prywatny** (stąd nazwa kryptografia z kluczem publicznym). Klucz publiczny może być swobodnie dystrybuowany otwartym kanałem i służy do szyfrowania (a także do weryfikowania podpisu). Klucz prywatny musi być utrzymywany w tajności i służy do deszyfrowania (lub tworzenia podpisu). 

Chronologicznie pierwszym kryptosystemem asymetrycznym był protokół wymiany kluczu Diffiego-Hellmana-Merkla. Służy on bezpiecznej wymiany danych, które mogą być wykorzystane jako tajne klucze kryptograficzne lub mogą być użyte do wyprodukowania kluczy. 

Najbardziej znanym kryptosystem z kluczem publicznym jest RSA (nazwa pochodzi od wynalazów: Rivest, Shamir i Adlemann). RSA umożliwia szyfrowanie danych jak również realizację podpisu cyfrowego. Bezpieczeństwo RSA opiera się na obliczeniowej trudności rozwiązania **problemu faktoryzacji liczb całkowitych złożonych**. 

In [1]:
## Funkcje pomocnicze 
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik 
    while a != 0:
        a, b = b % a, a
    return b

def gcdExtendedRecursion(a, b):  
    # przypadek trywialny
    if a == 0 :   
        return b,1,0             
    gcd,x1,y1 = gcdExtendedRecursion(b%a, a) #rekurencyjny Euklides      
   
    # sumowanie powrotne  
    y = x1 - (b//a) * y1  
    x = y1  
    
    return gcd,x,y 

def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m 
    # czyli (a*x) % m =1 

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny 

    # Rozszerzony algorytm Euklidesa 
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego 
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m


a, b = 154,801
g, x, y = gcdExtendedRecursion(a, b)  
print("gcd(", a , "," , b, ") = ", g, x, y)
print("Element odwrotny do ", a, "modulo", b, ":", y%b)

print(findModInverse(154,801))

gcd( 154 , 801 ) =  1 5 -26
Element odwrotny do  154 modulo 801 : 775
775


## Generowanie kluczy w kryptosystemie RSA

### 1. Losujemy dwie duże liczby pierwsze 
Potrzebujemy dwóch liczb pierwszych o naprawdę dużych rozmiarach - 2048 bitów obecnie uważa się niezbyt bezpieczny wybór. 4096 bitów jest z kolei wielkością nieco kłopotliwą w użytkowaniu. 
#### Skąd wziąć liczbę pierwszą? 
**Wylosować i sprawdzić czy jest pierwsza!**


Test probabilistyczny, np. Rabina-Millera. 

In [29]:
import random
def isPrime3(n, k):
    if n < 2: return False
    if n < 4: return True
    if n % 2 == 0: return False    # speedup

    s = 0
    d = n-1
    while d % 2 == 0:
        s += 1
        d //= 2

    for i in range(k):
        a = random.randrange(2, n-2)    # 2 <= a <= n-2
        x = pow(a,d,n)
        if (x != 1) and (x!= (n-1)):  #x^d przystaje 1 mod n (warunek silne pseudopierwszości) testujemy nowego swiadka
            for j in range(1,s):
                if (x== n-1): break 
                x = (x**2) % n
            if x != n-1: return False
    return True    

for i in range(65500,65600):
    if(isPrime3(i,10)): print(i)

65519
65521
65537
65539
65543
65551
65557
65563
65579
65581
65587
65599


In [390]:
def isPrimeFLT(p, rep):
    def gcd(a, b):
        while b:
            a, b = b, a%b
        return a
    
    if p == 2:
        return True
    elif p % 2 == 0:
        return False

    for _ in range(rep):
        a = random.randrange(1, p-1)
        while gcd(a, p) != 1:
            a = random.randrange(1, p-1)

        if pow(a, p-1, p) != 1:
            return False
    return True

for i in range(65500,65600):
    if(isPrimeFLT(i,10)): print(i)

65519
65521
65537
65539
65543
65551
65557
65563
65579
65581
65587
65599


## Zadanie
1. Napisz funkcję generującą liczbę pierwszą o określonej długości w bitach. 

In [307]:
#napisz funkcję generującą liczbę pierwszą 
def generatePrime(keysize):
    while True:   
        num = random.randint(2**(keysize-1),2**keysize)
        if(isPrime3(num, 6)):
            return num
    
    return num

generatePrime(1024)

108165530388592834119481938396357134046707518315316748431496824951771730313950243351521138017848275374177509737451769748691965300740108564779354698474962278385863922926733195816540026099852072954952034356402161883288075799813281330515356006186221913177885126445613647842557148512637680753590674610667024196839

## Zastanów się 
1. Znajdź zestawienie liczb silnie pseudopierwszych i sprawdź działanie (pojedynczego!) testy R-M dla takich liczb
2. Ile testów R-M należy przeprowadzić aby osiągnąć bezpieczny poziom pewności testu

### 2. Obliczamy składniki kluczy 
1. Wybieramy dwie duże liczby pierwsze $p$ i $q$
2. Pierwszym składnikiem klucza jest moduł $n$ $n=p \times q$ 
3. Poszukujemy wykładnika publicznego $e$, który jest względnie pierwszy z $(p-1)\cdot (q-1)$ (czasami używane jest w miejscu pojęcie tocjentu lub funkcji Eulera: $\phi(n) = \phi(p)\cdot \phi(q) = (p − 1)·(q − 1)$)
4. Poszukujemy wykładnika prywatnego $d$, które jest odwrotnością $e\ (mod\ (p-1)\cdot (q-1))$: $de \equiv 1  (mod\ (p-1)\cdot (q-1))$
5. Kluczem publiczny jest para $(n, e)$, kluczem prywatnym jest para $(n, d)$.

## Zadanie 

1. Napisz funkcję generującą klucze RSA o ustalonym rozmiarze

In [4]:
import random, sys, os

def generateKey(keySize):
    p = generatePrime(keySize)
    q = generatePrime(keySize)
    n = p * q
    euler_t = (p-1) * (q-1)
    e = 65537
    d = findModInverse(e, euler_t)
     
    publicKey = (n, e)
    privateKey = (n, d)
    
    return publicKey, privateKey

print('Generujemy klucze publiczny i prywatny')
print(generateKey(32))

Generujemy klucze publiczny i prywatny
((9938689558956413557, 65537), (9938689558956413557, 7176535479191729573))


## Zadanie 

Napisz funkcje implementujące szyfrowanie i deszyfrowanie RSA (tzw. podręcznikowe)

### Szyfrowanie RSA 
Operacja szyfrowania: $c=m^e (mod\ n)$

In [5]:
def encrypt(message, modulus, exp):
    message_encrypted = [pow(ord(char), exp, modulus) for char in message]
    return message_encrypted

### Deszyfrowanie RSA 
Operacja szyfrowanie $m = c^d (mod\ n)$

In [6]:
def decrypt(message_encrypted, modulus, exp):
    message_ascii = [chr(pow(char, exp, modulus)) for char in message_encrypted]
    return ('').join(message_ascii)

In [7]:
public, private = generateKey(32)

In [8]:
msg_encrypted = encrypt("ASD", public[0], public[1])
print(msg_encrypted)
msg_decrypted = decrypt(msg_encrypted, private[0], private[1])
print(msg_decrypted)

[1781006161437578285, 751502708826084011, 6553864227402830429]
ASD


## Zastanów się
1. Sprawdź działanie powyższej implementacji dla różnych wielkości klucza (podawane podczas generowania kluczy). 
2. Poszukaj informacji o trybie podręcznikowym RSA (*textbook RSA encryption*). Na czym polega? Jakie są jego wady i zalety? 


## Zadanie 
Zademonstruj deformowalność podręcznikowego szyfrowania RSA